# Matcher 
### Last Modified: July 14, 2020

To run, press SHIFT+ENTER on each cell. <br>
To get to next cell, press the down arrow after clicking on a blank spot of the cell. <br>
<b> Likewise, you can click on "Restart & Run All" under the Kernel menu (may need to re-run cells with dropdown boxes)</b><br>
Cells can be toggled on and off by running the cell preceding this message.
## If asked to "Try and re-run cell", then click on a blank spot of the cell until no dropdown box or button is selected and press SHIFT+ENTER
Go to https://docs.google.com/spreadsheets/d/1a2Ewf1T363tSrwSay7xDs4WMiaxqb2CwXgJzkvXX700 to view Matcher Sheet

In [ ]:
# Widgets
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
from ipywidgets import Layout, interact

# Imports
import pandas as pd
import numpy as np
import math

import io
import requests

import time

import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Variables
scope = "https://spreadsheets.google.com/feeds"
credentials = ServiceAccountCredentials.from_json_keyfile_name("matcher-272116-8801bce55bcb.json", scope)
gs = gspread.authorize(credentials)

matcher_sheet_url = "https://docs.google.com/spreadsheets/d/1a2Ewf1T363tSrwSay7xDs4WMiaxqb2CwXgJzkvXX700"

# Toggle Code Button
HTML('''<script>
code_show=true;
function code_toggle() {
    if (code_show){
    $('div.input').hide();
    } else {
    $('div.input').show();
    }
    code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the code."></form>
''')

# Journey Data

In [ ]:
# Creates matcher sheet update button and corresponding output
update_matcher_sheet_button = widgets.Button(description = "Update Matcher Sheet at 14:00 GMT", layout = Layout(width = "250px"))
out_update_matcher_sheet = widgets.Output()

def update_matcher_sheet(_) :
    with out_update_matcher_sheet :
        # Clears printed output
        clear_output()
        
        # Tells you how long it took to run
        # Initializes start time
        start_time = time.time()
        
        # Opens and clears Matcher Sheet's Journey worksheet
        express_guaranteed_sheet = gs.open_by_url(matcher_sheet_url).worksheet("Journeys")
        express_guaranteed_sheet.clear()
        
        # Updates Journey worksheet with cyfe board
        express_guaranteed_sheet.update("A1", '=importdata("https://www.anyvan.com/board/cyfe/express_interest_expresses")', raw = False)
        
        # Waits for cell A1 to be the value "journey_id"
        while (express_guaranteed_sheet.acell("A1").value != "#N/A" and express_guaranteed_sheet.acell("A1").value != "journey_id") :
            time.sleep(5)
            
        # Copies cells to save journey
        express_guaranteed_sheet.update("A1:X", express_guaranteed_sheet.get_all_values(), raw = False)
        
        # Prints out how long it took to run
        print("Updated Matcher Sheet with journey data from Express Interest Cyfe Board")
        print("Took {0:.3f} secs to run".format(time.time() - start_time))
        # Prints if cyfe board was empty
        if express_guaranteed_sheet.acell("A1").value == "#N/A" :
            print("No journeys loaded")

# Sets matcher sheet update button with function
update_matcher_sheet_button.on_click(update_matcher_sheet)

# Displays widgets
widgets.VBox([update_matcher_sheet_button, out_update_matcher_sheet])

In [ ]:
# Creates data load dropdown box, data load button, and corresponding output
express_guaranteed = widgets.Dropdown(
    options = ["Select option", "Matcher Sheet", "Express Interest Cyfe Board (May be empty now)"],
    value = "Select option",
    description = "Load data from:",
    style = {"description_width":"initial"})

express_guaranteed_button = widgets.Button(description = "Load", layout = Layout(width = "80px"))
out_journey = widgets.Output()

# Allows data to be saved outside of function
data = None

def load_journey_data(_) :
    # Saves data outside of function
    global data
    
    with out_journey :
        # Clears printed output
        clear_output()
        print("Loading journeys", end = "\r")
        
        if express_guaranteed.value == "Matcher Sheet" :
            # Opens Matcher Sheet's Journey worksheet and saves the journey data with the first row being the header
            express_guaranteed_sheet = gs.open_by_url(matcher_sheet_url).worksheet("Journeys").get_all_values()
            data = pd.DataFrame(express_guaranteed_sheet[1:], columns = express_guaranteed_sheet[0])
            data = data.replace("", np.nan)
            print("{0} journeys loaded, {1} expressions    ".format(data["journey_id"].nunique(), len(data["name#reg"].dropna())))
        elif express_guaranteed.value == "Expresss Interest Cyfe Board (May be empty now)" :
            # Opens the cyfe board and gets the content
            url = "https://www.anyvan.com/board/cyfe/express_interest_expresses"
            s = requests.get(url).content
            try :
                # Saves the journey data
                data = pd.read_csv(io.StringIO(s.decode("utf-8"))).sort_values(by = ["start_end"], ascending = False)
                print("{0} journeys loaded, {1} expressions    ".format(data["journey_id"].nunique(), len(data["name#reg"].dropna())))
            except :
                data = None
                print("No journeys available")
        else :
            data = None
            print("Select one of the options")
                
# Sets load data button with function
express_guaranteed_button.on_click(load_journey_data)

# Displays widgets
widgets.VBox([widgets.HBox([express_guaranteed, express_guaranteed_button]), out_journey])

In [ ]:
# Creates journey info dropdown box
# Does this if data is loaded or displays "Try to re-run cell" if data is not loaded
if data is not None :
    journey_info = widgets.Dropdown(
        options = np.insert(np.array(data["journey_id"].sort_values().unique()), 0, "Select journey"),
        value = "Select journey",
        description = "Journey:",
        style = {"description_width":"initial"})
else :
    journey_info = widgets.Dropdown(
        options = ["Select journey", "Try to re-run cell"],
        value = "Select journey",
        description = "Journey:",
        style = {"description_width":"initial"})

# Creates journey info button and corresponding output
journey_info_button = widgets.Button(description = "Display Journey Info", layout = Layout(width = "160px"))
out_journey_info = widgets.Output()
    
def display_journey_info(_):
    with out_journey_info:
        # Clears printed output
        clear_output()
        
        # Makes sure an actual journey is selected
        if journey_info.value != "Select journey" and journey_info.value != "Try to re-run cell" and journey_info.value != None :
            # Gets all the journey data for expressing TPs (including no expression journeys)
            data_TP = data.where(data["journey_id"] == journey_info.value).dropna(how = "all")
            # Gets generic journey data
            journey = data_TP.iloc[0]
        
            # Formats start time in HH:MM for all start times
            try :
                start_time = float(journey["start_time"])
                start_hour = int(start_time * 24)
                start_min = int((start_time * 24 - start_hour) * 60)
                
                start_time = "{0:02d}:{1:02d}".format(start_hour, start_min)
            except :
                start_time = journey["start_time"]
        
            # Prints generic journey data information
            print(u"Journey ID: {0}\nTP Amount: \xa3{1}\nStart to End Distance: {2:.2f} km\nPostcode: {3}".format(journey["journey_id"], journey["tp_amount"], float(journey["start_end"]), journey["postcode"]))
            print("Start Time: {0}\nNumber of Men: {1}\nTail Lift: {2}\nMax Volume: {3:.2f} m^3\nMax Weight: {4} kg".format(start_time, journey["number_of_men"], int(journey["tail_lift"]) == 1, float(journey["max_m3"]), journey["max_kg"]))
            
            # Gets all the journey data for journeys with expressions
            data_TP = data_TP.dropna(subset = ["name#reg"])
            # Prints number of expressions
            print("\nNumber of Expressions: {0}".format(len(data_TP.index)))
            # If there are expressions then displays TP information
            if len(data_TP) > 0 :
                print("Express Interest by:")
                for _,TP in data_TP.iterrows():
                    # Formats TP express return for all tp_quote_return (some are nan)
                    try :
                        tp_quote_return = int(TP["tp_quote_return"]) == 1
                    except :
                        tp_quote_return = False
                        
                    # Prints generic expressing TP information
                    print("\nTP Name and Reg: {0}\nDistance to Start: {1:.2f} km\nDistance to End: {2:.2f} km\nReturn: {3}".format(TP["name#reg"], float(TP["dist_to_start"]), float(TP["dist_to_end"]), tp_quote_return))
        
# Sets the journey info button with function
journey_info_button.on_click(display_journey_info)

# Displays widgets
widgets.VBox([widgets.HBox([journey_info, journey_info_button]), out_journey_info])

# TPs

In [ ]:
# Creates load TPs button and corresponding output
TPs_button = widgets.Button(description = "Load TPs", layout = Layout(width = "80 px"))
out_TPs = widgets.Output()

# Allows TPs to be saved outside of function
TPs = None

def load_TPs(_) :
    # Saves TPs outside of function
    global TPs
    
    with out_TPs :
        # Clears printed output
        clear_output()
        print("Loading TPs", end = "\r")
        
        # Returns from Return Google Sheet
        url = "https://docs.google.com/spreadsheets/d/1_syJVv1Uni8oSa9BhtAjVcl2EZYZmJn03VKg9vofcD4/edit#gid=0"

        return_journey_converter_sheet = gs.open_by_url(url).worksheet("Input").get_all_values()[15:]
        returns = pd.DataFrame(return_journey_converter_sheet[1:], columns = return_journey_converter_sheet[0])
        returns = returns.drop(columns = returns.columns[16:])
        returns["nickname"].replace("", float("NaN"), inplace = True)
        returns = returns.dropna()

        # Providers From TP Log Providers Google Sheet
        # Volume sizes based on vehicle
        sizes = {"Luton Van" : 20, 
                 "LWB up to 4m" : 10.5,
                 "MWB up to 3m" : 7.5,
                 "Not applicable" : 12,
                 "XLWB up 4m+" : 14.5,
                 "SWB up to 2.4m" : 5.5,
                 "Small Van" : 8}

        url = "https://docs.google.com/spreadsheets/d/1t0L9sgsUt-BT7wJWBNZ4fNY-Bo5GNytXP1wmkejFVw0/edit#gid=1205069010"

        TP_log_providers_sheet = gs.open_by_url(url).worksheet("format2").get_all_values()[7:]
        TP_log_providers = pd.DataFrame(TP_log_providers_sheet[1:], columns = TP_log_providers_sheet[0])
        storage_TP_reg = ["AVStore #YN19FHD", "AVStore #YN19FHF", "1London #YN19FHF", "DWB91 #YN19FHD", "AggyPat #YN19FHF"]
        providers = TP_log_providers.where(np.logical_and(TP_log_providers["vehicle"] != "N/A", ~TP_log_providers["TP & REG"].isin(storage_TP_reg))).dropna(how = "all").reset_index(drop = True).drop(columns = TP_log_providers.columns[-1])
        date = providers.columns[-1]

        # Initiliazes provider TP info
        providers["Size"] = [sizes[vehicle] for vehicle in providers["vehicle"]]
        # Artificially inflates TP max distance if code ends with "N"
        providers["National?"] = [int(math.ceil(55 + (100 - float(nuts2c)) / 2 + (2000 if code.endswith("N") else 0))) for nuts2c,code in zip(providers["nuts2c"], providers[date])]
        providers["1Man"] = [1 if (code.startswith("1") or code.startswith("O")) else 0 for code in providers[date]]
        providers["2Man"] = [2 if (code.startswith("2") or code.startswith("O")) else 0 for code in providers[date]]
        providers["returns"] = [1 if returns["name#reg"].str.contains(TP).any() else 0 for TP in providers["TP & REG"]]
        providers["guaranteed"] = [1 if isreturn == 1 else -1 for isreturn in providers["returns"]]
        providers["special code"] = ["2man-TL-LDN-cc" if (TwoMan == 2 and TL == "1") else "TL-LDN-cc" if (TL == "1") else "2man-LDN-cc" if (TwoMan == 2) else "LDN-cc" for TwoMan,TL in zip(providers["2Man"], providers["TL"])]
        providers["capacity"] = [16 if size >= 16 else size for size in providers["Size"]]
        
        providers = providers.sort_values(by = ["TP & REG"]).reset_index(drop = True)
        providers["index"] = providers.index

        # Expressing TPs from journey data if there is journey data available
        columns = ["TP", "lat", "lng", "expressions", "guaranteed", "boooked"]
        expressing_TPs = pd.DataFrame(columns = columns)

        if data is not None :
            for TP in data["name#reg"].dropna().unique() :
                data_TP = data.where(data["name#reg"] == TP).dropna(how = "all")
                lat = round(float(data_TP.iloc[0]["veh_lat"]), 2)
                lng = round(float(data_TP.iloc[0]["veh_lng"]), 2)
                expressions = len(data_TP.index)
                guaranteed = 1 if (providers["TP & REG"].str.contains(TP).any() or returns["name#reg"].str.contains(TP).any()) else 0
                booked = 1 if returns["name#reg"].str.contains(TP).any() else 0
                expressing_TPs = expressing_TPs.append(pd.DataFrame([[TP, lat, lng, expressions, guaranteed, booked]], columns = columns))
        else :
            print("No journeys available, please load data")
            print("No Express Interest TPs\n")

        expressing_TPs = expressing_TPs.sort_values(by = ["TP"]).reset_index(drop = True)
        expressing_TPs["index"] = expressing_TPs.index

        # Combine TPs from providers and expressions
        TP_namereg = np.append(providers["TP & REG"], expressing_TPs["TP"])
        TP_nickname = [name.split(" ")[0] for name in TP_namereg]
        TP_lat = np.append(providers["Lat"], expressing_TPs["lat"])
        TP_lng = np.append(providers["Lng"], expressing_TPs["lng"])
        TP_TL = np.append(providers["TL"], expressing_TPs["expressions"])
        TP_capacity = np.append(providers["Size"], np.full(len(expressing_TPs.index), np.nan))
        TP_maxkm = np.append(providers["National?"], np.full(len(expressing_TPs.index), np.nan))
        TP_1man = np.append(providers["1Man"], np.full(len(expressing_TPs.index), np.nan))
        TP_2man = np.append(providers["2Man"], np.full(len(expressing_TPs.index), np.nan))
        TP_guaranteed = np.append(providers["guaranteed"], expressing_TPs["guaranteed"])
        TP_provider = np.append(np.ones(len(providers.index), dtype = bool), np.zeros(len(expressing_TPs.index), dtype = bool))

        columns = ["TP", "nickname", "lat", "lng", "TL", "capacity", "maxkm", "1man", "2man", "guaranteed", "provider"]
        TPs = pd.DataFrame(zip(TP_namereg, TP_nickname, TP_lat, TP_lng, TP_TL, TP_capacity, TP_maxkm, TP_1man, TP_2man, TP_guaranteed, TP_provider), columns = columns)
        TPs = TPs.where(np.logical_and(TPs["guaranteed"] <= 0, ~TPs["TP"].duplicated())).dropna(how = "all").sort_values(by = ["provider", "TP"], ascending = [False, True])
        
        print("{0} TPs loaded, {1} Guaranteed TPs      ".format(len(TPs["TP"].index), len(TPs.where(TPs["guaranteed"] < 0).dropna(how = "all"))))
    
# Sets load TPs button with function
TPs_button.on_click(load_TPs)

# Displays widgets
widgets.VBox([TPs_button, out_TPs])

In [ ]:
# Creates TP info dropdown box
# Does this if TPs are loaded or displays "Try to re-run cell" if TPs are not loaded
if TPs is not None :
    TP_info = widgets.Dropdown(
        options = np.insert(np.array(TPs["TP"]), 0, "Select TP"),
        value = "Select TP",
        description = "TP:",
        style = {"description_width":"initial"})
else :
    TP_info = widgets.Dropdown(
        options = ["Select TP", "Try to re-run cell"],
        value = "Select TP",
        description = "TP:",
        style = {"description_width":"initial"})
    
# Creates TP info button and corresponding output
TP_info_button = widgets.Button(description = "Display TP Info",layout = Layout(width = "160px"))
out_TP_info = widgets.Output()

def display_TP_info(_) :
    with out_TP_info :
        # Clears printed output
        clear_output()
        
        # Makes sure an actual TP is selected
        if TP_info.value != "Select TP" and TP_info.value != "Try to re-run cell" and TP_info.value != None :
            # Gets the TP information
            TP = TPs.where(TPs["TP"] == TP_info.value).dropna(how = "all").iloc[0]
            
            # Prints the TP information
            print("TP Name and Reg: {0}\nTail Lift: {1}\nNumber of Men: {2}".format(TP["TP"], (int(TP["TL"]) == 1) if TP["provider"] == 1 else "Unknown", "1man" if TP["1man"] == 1 else "2man"))
            print("Vehicle Volume: {0} m^3\nMax Distance: {1} km\nGuaranteed: {2}\nProvider: {3}".format(TP["capacity"] if not np.isnan(TP["capacity"]) else "Unknown" , TP["maxkm"] if not np.isnan(TP["maxkm"]) else "Unknown", TP["guaranteed"] < 0, TP["provider"] == 1))
            
            # Displays TPs expressions if there is data
            if data is not None :
                # Gets TP's expressions
                data_TP = data.where(data["name#reg"] == TP["TP"]).dropna(how = "all")
                # Prints number of expressions
                print("\nNumber of Expressions: {0}".format(len(data_TP.index)))
                # If the TP expressed on journeys then displays them
                if len(data_TP.index) > 0 :
                    print("Express Interest on:")
                    for _,journey in data_TP.sort_values(by = "journey_id").iterrows():
                        # Formats TP express return for all tp_quote_return (some are nan)
                        try :
                            tp_quote_return = int(journey["tp_quote_return"]) == 1
                        except :
                            tp_quote_return = False
                            
                        # Prints generic journey info
                        print("\nJourney ID: {0}\nDistance to Start: {1:.2f} km\nDistance to End: {2:.2f} km\nReturn: {3}".format(journey["journey_id"], float(journey["dist_to_start"]), float(journey["dist_to_end"]), tp_quote_return))
        
# Sets TP info button with function
TP_info_button.on_click(display_TP_info)

# Displays widgets
widgets.VBox([widgets.HBox([TP_info, TP_info_button]), out_TP_info])

# Calculate IArray

In [ ]:
# Creates calculate iarray button and corresponding output
iarray_button = widgets.Button(description = "Calculate IArray", layout = Layout(width = "150px"))
out_iarray = widgets.Output()

# Defines distance calculation function
def distance_funct(start_lat, start_lng, lat, lng) :
    # Not sure why it is multiplied by 6371
    return 2 * 6371 * np.arcsin(np.sqrt(np.sin((start_lat - lat) * np.pi / 360) ** 2 
                                           + np.cos(start_lat * np.pi / 180) 
                                           * np.cos(lat * np.pi / 180) 
                                           * np.sin((start_lng - lng) * np.pi / 360) ** 2))

# Allows iarray and journeys_data to be saved outside of function
iarray = None
journeys_data = None

def calc_iarray(_) :
    # Saves iarray and journeys_data outside of function
    global iarray, journeys_data
    
    with out_iarray :
        # Clears printed output
        clear_output()
        
        # Runs only if there is data
        if data is not None and TPs is not None :
            # Tells you how long it took to run
            # Initializes start time
            start_time = time.time()

            # Gets list of unique journey ids
            journeys = data["journey_id"].unique()

            # Initializes journeys_data and iarray
            columns = ["journey_id", "tp_amount", "vat", "vat_pef_multiplier", "start_end"]
            journeys_data = pd.DataFrame(columns = columns)
            iarray = np.zeros((len(TPs.index), len(journeys)), dtype = float)

            # Sets vat impact for journeys data's vat_pef_multiplier
            vat_impact = 1000

            # Creates iarray progress bar
            iarray_progress = widgets.IntProgress(
                value = 0,
                min = 0,
                max = len(journeys) - 1,
                step = 1,
                description = "Calculating",
                bar_style = "success",
                orientation = "horizontal",
                layout = Layout(width = "40%"),
                style = {"description_width":"initial"}
            )

            # Displays iarray progress bar
            display(widgets.HBox([iarray_progress]))

            for i,journey_id in enumerate(journeys) :
                # Gets all expressions and generic journey data
                journey = data.where(data["journey_id"] == journey_id).dropna(how = "all")
                # Saves generic journey data to journeys_data
                journeys_data = journeys_data.append(pd.DataFrame([[str(int(journey["journey_id"].iloc[0])), journey["tp_amount"].iloc[0], float(journey["tp_amount"].iloc[0]) / 6 / 1.2, 1 - np.tanh(float(journey["tp_amount"].iloc[0]) / 6 / 1.2 / vat_impact), journey["start_end"].iloc[0]]], columns = columns))

                # Calculates iarray value for each TP for the journey
                for j in range(len(TPs.index)) :
                    # If TP is guaranteed
                    if int(TPs["guaranteed"].iloc[j]) == -1 :
                        # If guaranteed TP expressed on journey
                        if TPs["TP"].iloc[j] in journey["name#reg"].values :
                            # Gets guaranteed TPs expression
                            journey_subset = journey.where(journey["name#reg"] == TPs["TP"].iloc[j]).dropna(how = "all").iloc[0]
                            # Sets iarray value to 5 if min_dist for TP is >= 5
                            if float(journey_subset["min_dist"]) >= 5 :
                                iarray[j, i] = 5
                            # Else sets iarray value to min_dist for TP
                            else :
                                iarray[j, i] = float(journey_subset["min_dist"])
                        # Else if guaranteed TP didn't express on journey
                        else :
                            # If journey volume <= TP capacity
                            if float(journey["max_m3"].iloc[0]) <= float(TPs["capacity"].iloc[j]) :
                                # If journey start to end distance <= TP max distance
                                if float(journey["start_end"].iloc[0]) <= float(TPs["maxkm"].iloc[j]) :
                                    # If journey number of men the same as TP number of men
                                    # Allows for TP to be interested in both 1man, 2man, or both
                                    if int(journey["number_of_men"].iloc[0]) == int(TPs["1man"].iloc[j]) or int(journey["number_of_men"].iloc[0]) == int(TPs["2man"].iloc[j]) :
                                        # If journey tail lift requirement is met by TP tail lift ability
                                        if int(journey["tail_lift"].iloc[0]) <= int(TPs["TL"].iloc[j]) :
                                            # Calculates distance betwen journey start location and TP location
                                            distance = distance_funct(float(journey["start_lat"].iloc[0]), float(journey["start_lng"].iloc[0]), float(TPs["lat"].iloc[j]), float(TPs["lng"].iloc[j]))
                                            # If distance <= TP max distance 
                                            # Accounts for TP max distance being artificially inflated
                                            if distance <= (float(TPs["maxkm"].iloc[j]) if float(TPs["maxkm"].iloc[j]) < 2000 else float(TPs["maxkm"].iloc[j]) - 2000) :
                                                # Sets iarray value to distance
                                                iarray[j, i] = distance
                    # Else if TP is not guaranteed and TP expressed on journey
                    elif TPs["TP"].iloc[j] in journey["name#reg"].values :
                        # Sets iarray value to TP min_dist
                        iarray[j, i] = float(journey.where(journey["name#reg"] == TPs["TP"].iloc[j]).dropna(how = "all")["min_dist"].iloc[0])
                    # Not shown but, else if TP not guaranteed and TP not expressed than sets iarray value to 0
                # Updates iarray progress bar
                iarray_progress.value = i

            # Prints how long it took to calculate iarray in seconds
            print("Took {0:.3f} secs to run".format(time.time() - start_time))
        # Else if there is no data then cancel
        else :
            print("Load data and TPs First")
            return

        # Finalizes journeys_data
        journeys_data = journeys_data.reset_index(drop = True)
    
# Sets calculate iarray button with function
iarray_button.on_click(calc_iarray)

# Displays widgets
widgets.VBox([iarray_button, out_iarray])

In [ ]:
# Creates journey and TP dropdown box
# Does this if data is loaded or displays "Try to re-run cell" if data is not loaded
if journeys_data is not None and TPs is not None :
    journey_menu = widgets.Dropdown(
        options = np.insert(np.array(journeys_data.sort_values(by = "journey_id")["journey_id"]), 0, "Select journey"),
        value = "Select journey",
        description = "Journey:",
        style = {"description_width":"initial"})
    
    TP_menu = widgets.Dropdown(
        options = np.insert(np.array(TPs["TP"]), 0, "Select TP"),
        value = "Select TP",
        description = "TP:",
        style = {"description_width":"initial"})
else :
    journey_menu = widgets.Dropdown(
        options = ["Select journey", "Try to re-run cell"],
        value = "Select journey",
        description = "Journey:",
        style = {"description_width":"initial"})
    
    TP_menu = widgets.Dropdown(
        options = ["Select TP", "Try to re-run cell"],
        value = "Select TP",
        description = "TP:",
        style = {"description_width":"initial"})

# Creates find journey and find TP buttons as well as corresponding output
journey_button = widgets.Button(description = "Find", layout = Layout(width = "80px"))
TP_button = widgets.Button(description = "Find", layout = Layout(width = "80px"))
out_display = widgets.Output()

def journey_display(_) :
    # Sets TP dropdown box to "Select TP"
    TP_menu.value = "Select TP"
    
    with out_display :
        # Clears printed output
        clear_output()
        
        # Makes sure an actual journey is selected
        if journey_menu.value != "Select journey" and journey_menu.value != "Try to re-run cell" and journey_menu.value != None:
            print("Journey: {0}\n".format(journey_menu.value))
            # Prints iarray value for each TP for the journey
            for TP, iarray_val in list(zip(TPs["TP"], iarray.transpose()[np.argwhere(np.array(journeys_data["journey_id"]) == journey_menu.value)][0][0])) :
                print("{0:20} -> {1}".format(TP, iarray_val))

def TP_display(_) :
    # Sets journey dropdown box to "Select journey"
    journey_menu.value = "Select journey"
    
    with out_display :
        # Clears printed output
        clear_output()
        
        # Makes sure an actual TP is selected
        if TP_menu.value != "Select TP" and TP_menu.value != "Try to re-run cell" and TP_menu.value != None:
            print("TP: {0}\n".format(TP_menu.value))
            # Prints iarray value for each journey for the TP
            for journey, iarray_val in list(zip(journeys_data["journey_id"], iarray[np.argwhere(np.array(TPs["TP"]) == TP_menu.value)][0][0])) :
                print("{0:20} -> {1}".format(journey, iarray_val))

# Sets find journey and find TP buttons to their corresponding functions
journey_button.on_click(journey_display)
TP_button.on_click(TP_display)

# Displays widgets
widgets.VBox([widgets.HBox([journey_menu, journey_button, TP_menu, TP_button]), out_display])

In [ ]:
# Creates journey and TP dropdown box for both normal edit and remove entirely
# Does this if data is loaded or displays "Try to re-run cell" if data is not loaded
if journeys_data is not None and TPs is not None :
    journey_menu_edit = widgets.Dropdown(
        options = np.insert(np.array(journeys_data.sort_values(by = "journey_id")["journey_id"]), 0, "Select journey"),
        value = "Select journey",
        description = "Journey:",
        style = {"description_width":"initial"})
    
    journey_menu_remove = widgets.Dropdown(
        options = np.insert(np.array(journeys_data.sort_values(by = "journey_id")["journey_id"]), 0, "Select journey"),
        value = "Select journey",
        description = "Journey:",
        style = {"description_width":"initial"})
    
    TP_menu_edit = widgets.Dropdown(
        options = np.insert(np.array(TPs["TP"]), 0, "Select TP"),
        value = "Select TP",
        description = "TP:",
        style = {"description_width":"initial"})
    
    TP_menu_remove = widgets.Dropdown(
        options = np.insert(np.array(TPs["TP"]), 0, "Select TP"),
        value = "Select TP",
        description = "TP:",
        style = {"description_width":"initial"})
else :
    journey_menu_edit = widgets.Dropdown(
        options = ["Select journey", "Try to re-run cell"],
        value = "Select journey",
        description = "Journey:",
        style = {"description_width":"initial"})
    
    journey_menu_remove = widgets.Dropdown(
        options = ["Select journey", "Try to re-run cell"],
        value = "Select journey",
        description = "Journey:",
        style = {"description_width":"initial"})
    
    TP_menu_edit = widgets.Dropdown(
        options = ["Select TP", "Try to re-run cell"],
        value = "Select TP",
        description = "TP:",
        style = {"description_width":"initial"})
    
    TP_menu_remove = widgets.Dropdown(
        options = ["Select TP", "Try to re-run cell"],
        value = "Select TP",
        description = "TP:",
        style = {"description_width":"initial"})
    
# Creates iarray edit float number box
iarray_edit_box = widgets.FloatText(
    value = 0,
    description = "New IArray value:",
    style = {"description_width":"initial"})

# Creates update iarray and iarray remove buttons and corresponding output
iarray_edit_button = widgets.Button(description = "Update IArray", layout = Layout(width = "115px"))
iarray_remove_TP_button = widgets.Button(description = "Remove TP from IArray", layout = Layout(width = "200px"))
iarray_remove_journey_button = widgets.Button(description = "Remove Journey from IArray", layout = Layout(width = "200px"))
out_edit = widgets.Output()

def iarray_edit(_) :
    # Sets journey and TP removal dropdown boxes to initial values
    journey_menu_remove.value = "Select journey"
    TP_menu_remove.value = "Select TP"
    
    # Does not clear printed output
    with out_edit :
        # Makes sure both a TP and a journey are selected and that they are an actual TP and an actual journey
        if journey_menu_edit.value == "Try to re-run cell" or TP_menu_edit.value == "Try to re-run cell" :
            print("Try to re-run cell\n")
        elif journey_menu_edit.value == "Select journey" or TP_menu_edit.value == "Select TP" :
            print("Select a journey and a TP\n")
        elif iarray_edit_box.value is None or iarray_edit_box.value < 0 :
            print("Select an appropriate value for the iarray edit\n")
        else :
            # Saves old iarray value
            old_iarray_value = iarray[np.argwhere(np.array(TPs["TP"]) == TP_menu_edit.value), np.argwhere(np.array(journeys_data["journey_id"]) == journey_menu_edit.value)][0][0]
            # Updates iarray with new value
            iarray[np.argwhere(np.array(TPs["TP"]) == TP_menu_edit.value), np.argwhere(np.array(journeys_data["journey_id"]) == journey_menu_edit.value)] = float(iarray_edit_box.value)
            # Prints out TP and journey information with old and new iarray value
            print("TP: {0}\nJourney: {1}\nOld IArray Value: {2}\nNew Iarray Value: {3}\n".format(TP_menu_edit.value, journey_menu_edit.value, old_iarray_value, iarray_edit_box.value))
            
def iarray_remove_TP(_) :
    # Sets journey and TP edit dropdown boxes as well as journey remove dropdown box to initial values
    journey_menu_edit.value = "Select journey"
    TP_menu_edit.value = "Select TP"
    journey_menu_remove.value = "Select journey"
    
    # Does not clear printed output
    with out_edit :
        # Makes sure a TP is selected and that they are an actual TP
        if TP_menu_remove.value == "Try to re-run cell" :
            print("Try to re-run cell\n")
        elif TP_menu_remove.value != "Select TP" :
            # Saves old iarray values
            old_iarray_values = iarray[np.argwhere(np.array(TPs["TP"]) == TP_menu_remove.value)][0][0]
            for i,old_iarray_value in enumerate(old_iarray_values) :
                # If the iarray value is non-zero
                if old_iarray_value > 0 :
                    # Updates iarray to 0
                    iarray[np.argwhere(np.array(TPs["TP"]) == TP_menu_remove.value), i] = 0
                    # Prints out TP and journey information with old iarray value
                    print("TP: {0}\nJourney: {1}\nOld IArray Value: {2}\nNew Iarray Value: 0\n".format(TP_menu_remove.value, journeys_data["journey_id"].iloc[i], old_iarray_value))
                    
def iarray_remove_journey(_) :
    # Sets journey and TP edit dropdown boxes as well as TP remove dropdown box to initial values
    journey_menu_edit.value = "Select journey"
    TP_menu_edit.value = "Select TP"
    TP_menu_remove.value = "Select TP"
    
    # Does not clear printed output
    with out_edit :
        # Makes sure a journey is selected and that it is an actual journey
        if journey_menu_remove.value == "Try to re-run cell" :
            print("Try to re-run cell\n")
        elif journey_menu_remove.value != "Select journey" :
            # Saves old iarray values
            old_iarray_values = iarray.transpose()[np.argwhere(np.array(journeys_data["journey_id"]) == journey_menu_remove.value)][0][0]
            for i,old_iarray_value in enumerate(old_iarray_values) :
                # If the iarray value is non-zero
                if old_iarray_value > 0 :
                    # Updates iarray to 0
                    iarray[i,np.argwhere(np.array(journeys_data["journey_id"]) == journey_menu_remove.value)] = 0
                    # Prints out TP and journey information with old iarray value
                    print("TP: {0}\nJourney: {1}\nOld IArray Value: {2}\nNew Iarray Value: 0\n".format(TPs["TP"].iloc[i], journey_menu_remove.value, old_iarray_value))
            
# Sets update iarray and iarray remove buttons with function
iarray_edit_button.on_click(iarray_edit)
iarray_remove_TP_button.on_click(iarray_remove_TP)
iarray_remove_journey_button.on_click(iarray_remove_journey)

# Displays widgets
widgets.VBox([widgets.HBox([journey_menu_edit, TP_menu_edit, iarray_edit_box, iarray_edit_button]), 
              widgets.HBox([journey_menu_remove, iarray_remove_journey_button]), 
              widgets.HBox([TP_menu_remove, iarray_remove_TP_button]),
              out_edit])

# Provider Stats and TP Score

In [ ]:
# Creates load providers stats and TP score button with corresponding output
provider_stats_TP_score_button = widgets.Button(description = "Load Provider Stats and Calculate TP Score", layout = Layout(width = "300px"))
out_provider_stats_TP_score = widgets.Output()

# Allows TP_pstats to be saved outside of function
TP_pstats = None

def load_stat_calc_score(_) :
    # Saves TP_pstats outside of function
    global TP_pstats
    
    with out_provider_stats_TP_score :
        # Clears printed output
        clear_output()
        
        # Makes sure TPs are already loaded
        if TPs is None :
            print("Load TPs First")
            return
        
        print("Loading provider stats", end = "\r")
        # Opens the provider stats and gets the content
        url = "https://www.anyvan.com/board/cyfe/express_interest_pstats"
        s = requests.get(url).content

        try :
            # Loads the provider stats
            pstats = pd.read_csv(io.StringIO(s.decode("utf-8")))
            print("Provider stats loaded                ")
        except :
            data = None
            print("No provider stats available")

        # Weights to use for various scores
        score_weight = {"rank_vat" : 10,
                        "rank_g" : 25,
                        "rank_wps" : 1,
                        "rank_rating" : 2,
                        "rank_dsw" : 1,
                        "rank_ssw" : 2,
                        "rank_exp" : 1}

        # Updates existing TPs with their provider stats
        TP_pstats = TPs.merge(pstats, how = "left", left_on = "nickname", right_on = "nickname")

        # Initializes scores for TPs with weights
        TP_pstats["express"] = [10 if guaranteed == -1 else TL for guaranteed, TL in zip(TP_pstats["guaranteed"], TP_pstats["TL"])]
        TP_pstats["rank_vat"] = TP_pstats["vat?"].rank(method = "min", ascending = False) / len(TP_pstats.index)
        TP_pstats["rank_g"] = TP_pstats["guaranteed"].rank(method = "min", ascending = False) / len(TP_pstats.index)
        TP_pstats["rank_wps"] = TP_pstats["wins per session"].rank(method = "min", ascending = False) / len(TP_pstats.index)
        TP_pstats["rank_rating"] = TP_pstats["avg rating"].rank(method = "min", ascending = True) / len(TP_pstats.index)
        TP_pstats["rank_dsw"] = TP_pstats["days_since_win"].rank(method = "min", ascending = True) / len(TP_pstats.index)
        TP_pstats["rank_ssw"] = TP_pstats["sessions_since_win"].rank(method = "min", ascending = True) / len(TP_pstats.index)
        TP_pstats["rank_exp"] = TP_pstats["express"].rank(method = "min", ascending = True) / len(TP_pstats.index)

        # Calculates overall weighted score
        TP_pstats["weighted_score"] = TP_pstats["rank_vat"] * score_weight["rank_vat"] + TP_pstats["rank_g"] * score_weight["rank_g"] + TP_pstats["rank_wps"] * score_weight["rank_wps"] + TP_pstats["rank_rating"] * score_weight["rank_rating"] + TP_pstats["rank_dsw"] * score_weight["rank_dsw"] + TP_pstats["rank_ssw"] * score_weight["rank_ssw"] + TP_pstats["rank_exp"] * score_weight["rank_exp"]
        # Saves overall max weighted score for weighted rank
        weighted_score_max = TP_pstats["weighted_score"].max()

        # Calculates weighted rank
        TP_pstats["weighted_rank"] = [0.75 if guaranteed == -1 else (1.5 - weighted_score / weighted_score_max) for guaranteed, weighted_score in zip(TP_pstats["guaranteed"], TP_pstats["weighted_score"])]

        print("TP Score calculated")
        
# Sets load providers stats and TP score button with function
provider_stats_TP_score_button.on_click(load_stat_calc_score)

# Displays widgets
widgets.VBox([provider_stats_TP_score_button, out_provider_stats_TP_score])

# MArray 
(Need to run again if updated IArray values)

In [ ]:
# Creates calculate marray button and corresponding output
marray_button = widgets.Button(description = "Calculate MArray", layout = Layout(width = "150px"))
out_marray = widgets.Output()

# Allows marray to be saved outside of function
marray = None

def calc_marray(_) :
    # Saves marray outside of function
    global marray
    
    with out_marray :
        # Clears printed output
        clear_output()
        
        # Makes sure iarray is already calculated
        if iarray is None :
            print("Calculate IArray First")
            return
        
        # Initializes marray
        marray = iarray.copy()

        for i in range(len(journeys_data.index)) :
            for j in range(len(TP_pstats.index)) :
                if iarray[j, i] > 0 :
                    # Sets marray value based on weighted rank and var_pef_multiplier
                    marray[j, i] = (iarray[j,i] + (100 if int(TP_pstats["guaranteed"].iloc[j]) >= 0 else 0)) * float(TP_pstats["weighted_rank"].iloc[j]) * (float(journeys_data["vat_pef_multiplier"].iloc[i]) if int(TP_pstats["vat?"].iloc[j]) == 0 else 1)
                    
        print("MArray calculated")
                    
# Sets calculate marray button with function
marray_button.on_click(calc_marray)

# Disiplays widgets
widgets.VBox([marray_button, out_marray])

In [ ]:
# Creates journey and TP dropdown box
# Does this if data is loaded or displays "Try to re-run cell" if data is not loaded
if journeys_data is not None and TP_pstats is not None :
    journey_menu_marray = widgets.Dropdown(
        options = np.insert(np.array(journeys_data.sort_values(by = "journey_id")["journey_id"]), 0, "Select journey"),
        value = "Select journey",
        description = "Journey:",
        style = {"description_width":"initial"})
    
    TP_menu_marray = widgets.Dropdown(
        options = np.insert(np.array(TP_pstats["TP"]), 0, "Select TP"),
        value = "Select TP",
        description = "TP:",
        style = {"description_width":"initial"})
else :
    journey_menu_marray = widgets.Dropdown(
        options = ["Select journey", "Try to re-run cell"],
        value = "Select journey",
        description = "Journey:",
        style = {"description_width":"initial"})
    
    TP_menu_marray = widgets.Dropdown(
        options = ["Select TP", "Try to re-run cell"],
        value = "Select TP",
        description = "TP:",
        style = {"description_width":"initial"})

# Creates find journey and find TP buttons with corresponding output
journey_button_marray = widgets.Button(description = "Find", layout = Layout(width = "80px"))
TP_button_marray = widgets.Button(description = "Find", layout = Layout(width = "80px"))
out_display_marray = widgets.Output()

def journey_display_marray(_) :
    # Sets TP dropdown box to "Select TP"
    TP_menu_marray.value = "Select TP"
    
    with out_display_marray :
        # Clears printed output
        clear_output()
        
        # Makes sure an actual journey is selected
        if journey_menu_marray.value != "Select journey" and journey_menu_marray.value != "Try to re-run cell" and journey_menu_marray.value != None:
            # Makes sure MArray is calculated
            if marray is None :
                print("Calculate MArray First")
                return
            
            print("Journey: {0}\n".format(journey_menu_marray.value))
            # Prints marray value for each TP for the journey
            for TP, marray_val in list(zip(TP_pstats["TP"], marray.transpose()[np.argwhere(np.array(journeys_data["journey_id"]) == journey_menu_marray.value)][0][0])) :
                print("{0:20} -> {1}".format(TP, marray_val))

def TP_display_marray(_) :
    # Sets journey dropdown box to "Select journey"
    journey_menu_marray.value = "Select journey"
    
    with out_display_marray :
        # Clears printed output
        clear_output()
        
        # Makes sure an actual TP is selected
        if TP_menu_marray.value != "Select TP" and TP_menu_marray.value != "Try to re-run cell" and TP_menu_marray.value != None:
            # Makes sure MArray is calculated
            if marray is None :
                print("Calculate MArray First")
                return
        
            print("TP: {0}\n".format(TP_menu_marray.value))
            # Prints marrray value for each journey for the TP
            for journey, marray_val in list(zip(journeys_data["journey_id"], marray[np.argwhere(np.array(TPs["TP"]) == TP_menu_marray.value)][0][0])) :
                print("{0:20} -> {1}".format(journey, marray_val))

# Sets find journey and find TP buttons with their corresponding functions
journey_button_marray.on_click(journey_display_marray)
TP_button_marray.on_click(TP_display_marray)

# Displays widgets
widgets.VBox([widgets.HBox([journey_menu_marray, journey_button_marray, TP_menu_marray, TP_button_marray]), out_display_marray])

# Matcher
(Need to run versions again if updated IArray values)<br>
### Matcher v1.0
Uses no journey sorting<br>
### Matcher v2.0
Sorts journeys based on ascending/descending number of possible matches<br>
### Matcher v3.0
Sorts journeys based on ascending/descending minimum MArray value<br>
### Matcher v4.0
Sorts journeys based on ascending/descending start to end distance value

In [ ]:
# Creates ascending/descending checkbox
ascending = widgets.Checkbox(
    value = False, 
    description = "Ascending (checked) or Descending (unchecked) ordering?",
    disabled = False,
    layout = Layout(width = "500px"))

# Allows matches and to be saved outside of function
matches = None

# Sets max number of iterations to do
# Updated to keep running until length of journeys_data_copy doesn't change after running through
number_iterations = 25

# Saves which was the last matcher run
last_matcher_run = ""
asc_desc = ""

# General Matcher
def matcher_v0(journeys_data_copy, matcher_progress, sorting, out_matcher):
    # Saves matches outside of function
    global matches
    
    with out_matcher :
        # Clears printed output
        clear_output()
        
        # Tells you how long it took to run
        # Initializes start time
        start_time = time.time()
        
        # Initializes matches
        matches = journeys_data.copy()
        matches["TP"] = np.empty(len(matches.index), dtype = object)
        
        # Copies marray and TP_pstats to be used in matcher
        marray_copy = marray.copy()
        TP_pstats_copy = TP_pstats.copy()

        # Displays matcher progress bar
        display(widgets.HBox([matcher_progress]))
        
        # Initializes iteration for both number of iterations
        iteration = 0
        # And decreasing number of journeys
        journeys_len_init = len(journeys_data_copy.index)
        journeys_len_end = 0
        
        while iteration < number_iterations and journeys_len_end - journeys_len_init < 0:
            # Updates iteration count
            iteration += 1
            
            # Saves initial size of journeys_data_copy
            journeys_len_init = len(journeys_data_copy.index)
            
            # Updates matcher progress bar between iterations
            matcher_progress.value = 0
            matcher_progress.max = len(journeys_data_copy["journey_id"]) - 1
            
            # Runs through each journey
            for journey_id in sorting(journeys_data_copy)["journey_id"] :
                # Prints how many journeys unallocated, how many TPs are left and what iteration we are on
                print("{0} Journeys Unallocated, {1} Guaranteed TP(s) Left, Iteration {2} of {3}      ".format(len(journeys_data_copy), len(TP_pstats_copy.where(TP_pstats_copy["guaranteed"] < 0).dropna(how = "all").index), iteration, number_iterations), end = "\r")
                
                # Gets journey information
                journey = journeys_data_copy.where(journeys_data_copy["journey_id"] == journey_id).dropna(how = "all")
                # Gets journey index for marray
                journey_index = journeys_data_copy.index[journeys_data_copy["journey_id"] == journey_id][0]
                # Gets marray values for journey
                marray_journey = marray_copy[:,journey_index]
                
                # If there is a possible match for the journey
                if len(marray_journey[np.nonzero(marray_journey)]) > 0 :
                    # Finds the TP with the minimum marray value
                    best_TP_index = np.argwhere(marray_journey == np.min(marray_journey[np.nonzero(marray_journey)]))[0][0]

                    # If there is only one TP possible for journey
                    if len(marray_journey[np.nonzero(marray_journey)]) == 1 :
                        # Matches TP to journey and saves their name
                        matches.loc[journeys_data.index[journeys_data["journey_id"] == journey_id][0], "TP"] = TP_pstats_copy.iloc[best_TP_index]["TP"]

                        # Removes journey from journeys_data_copy
                        journeys_data_copy = journeys_data_copy.drop(journey_index).reset_index(drop = True)
                        # Removes TP from TP_pstats_copy
                        TP_pstats_copy = TP_pstats_copy.drop(best_TP_index).reset_index(drop = True)
                        # Removes journey and TP from marray_copy
                        marray_copy = np.delete(marray_copy, best_TP_index, 0)
                        marray_copy = np.delete(marray_copy, journey_index, 1)
                    # Else if there is more than one possible TP for journey
                    else :
                        # Get marray values for TP
                        marray_TP = marray_copy[best_TP_index, :]
                        # Finds the journey with the minimum marray value
                        best_journey_index = np.argwhere(marray_TP == np.min(marray_TP[np.nonzero(marray_TP)]))[0][0]

                        # If the journey is the best for the TP and the TP is the best for the journey
                        if journey_index == best_journey_index :
                            # Matches TP to journey and saves their name
                            matches.loc[journeys_data.index[journeys_data["journey_id"] == journey_id][0], "TP"] = TP_pstats_copy.iloc[best_TP_index]["TP"]

                            # Removes journey from journeys_data_copy
                            journeys_data_copy = journeys_data_copy.drop(journey_index).reset_index(drop = True)
                            # Removes TP from TP_pstats_copy
                            TP_pstats_copy = TP_pstats_copy.drop(best_TP_index).reset_index(drop = True)
                            # Removes journey and TP from marray_copy
                            marray_copy = np.delete(marray_copy, best_TP_index, 0)
                            marray_copy = np.delete(marray_copy, journey_index, 1)

                # Updates matcher progress bar
                matcher_progress.value += 1
            
            # And end size of journeys_data_copy
            journeys_len_end = len(journeys_data_copy.index)
        
        # Prints end number of iterations and total time to run matcher
        print("\nFinished Matcher in {0} iterations and {1:.3f} sec".format(iteration, time.time() - start_time))

# No sorting
def matcher_v1(_):
    # Saves last_matcher_run outside of function
    global last_matcher_run, asc_desc
    
    # Makes sure marray is already created
    if marray is None :
        with out_matcher_v1 :
            clear_output()
            print("Calculate MArray First")
            return
    
    # Sets last matcher run
    last_matcher_run = "Matcher v1.0"
    asc_desc = ""
    
    # Copies journeys_data
    journeys_data_copy = journeys_data.copy()
    
    # Creates matcher progress bar
    matcher_progress_v1 = widgets.IntProgress(
        value = 0,
        min = 0,
        max = len(journeys_data_copy["journey_id"]) - 1,
        step = 1,
        description = "Calculating v1.0",
        bar_style = "success",
        orientation = "horizontal",
        layout = Layout(width = "40%"),
        style = {"description_width":"initial"}
    )
    
    # Sorting function
    def sorting(array) :
        return array
    
    matcher_v0(journeys_data_copy, matcher_progress_v1, sorting, out_matcher_v1)

# Sorts journeys based on decreasing number of possible journeys
def matcher_v2(_):
    # Saves last_matcher_run outside of function
    global last_matcher_run, asc_desc
    
    # Makes sure marray is already created
    if marray is None :
        with out_matcher_v2 :
            clear_output()
            print("Calculate MArray First")
            return
    
    # Sets last matcher run
    last_matcher_run = "Matcher v2.0"
    asc_desc = "Ascending Ordering" if ascending.value else "Descending Ordering"
    
    # Copies journeys_data
    journeys_data_copy = journeys_data.copy()
    # Counts number of possible matches for each journey
    number_possible = np.zeros(len(journeys_data_copy))
    for i in range(len(number_possible)) :
        number_possible[i] = marray[:,i][np.nonzero(marray[:,i])].size
    journeys_data_copy["number_possible"] = number_possible
    
    # Creates matcher progress bar
    matcher_progress_v2 = widgets.IntProgress(
        value = 0,
        min = 0,
        max = len(journeys_data_copy["journey_id"]) - 1,
        step = 1,
        description = "Calculating v2.0",
        bar_style = "success",
        orientation = "horizontal",
        layout = Layout(width = "40%"),
        style = {"description_width":"initial"}
    )
    
    # Sorting function
    def sorting(array) :
        return array.sort_values(by = ["number_possible"], ascending = ascending.value)
    
    matcher_v0(journeys_data_copy, matcher_progress_v2, sorting, out_matcher_v2)

# Sorts journeys based on decreasing minimum MArray value
def matcher_v3(_):
    # Saves last_matcher_run outside of function
    global last_matcher_run, asc_desc
    
    # Makes sure marray is already created
    if marray is None :
        with out_matcher_v3 :
            clear_output()
            print("Calculate MArray First")
            return
    
    # Sets last matcher run
    last_matcher_run = "Matcher v3.0"
    asc_desc = "Ascending Ordering" if ascending.value else "Descending Ordering"
    
    # Copies journeys_data
    journeys_data_copy = journeys_data.copy()
    # Finds minimum MArray value for each journey
    min_marray = np.zeros(len(journeys_data_copy))
    for i in range(len(min_marray)) :
        try :
            min_marray[i] = np.min(marray[:,i][np.nonzero(marray[:,i])])
        except :
            min_marray[i] = 0
    journeys_data_copy["min_marray"] = min_marray
    
    # Creates matcher progress bar
    matcher_progress_v3 = widgets.IntProgress(
        value = 0,
        min = 0,
        max = len(journeys_data_copy["journey_id"]) - 1,
        step = 1,
        description = "Calculating v3.0",
        bar_style = "success",
        orientation = "horizontal",
        layout = Layout(width = "40%"),
        style = {"description_width":"initial"}
    )
    
    # Sorting function
    def sorting(array) :
        return array.sort_values(by = ["min_marray"], ascending = ascending.value)
    
    matcher_v0(journeys_data_copy, matcher_progress_v3, sorting, out_matcher_v3)
    
# Sorts journeys based on decreasing start to end distance value
def matcher_v4(_):
    # Saves last_matcher_run outside of function
    global last_matcher_run, asc_desc
    
    # Makes sure marray is already created
    if marray is None :
        with out_matcher_v4 :
            clear_output()
            print("Calculate MArray First")
            return
    
    # Sets last matcher run
    last_matcher_run = "Matcher v4.0"
    asc_desc = "Ascending Ordering" if ascending.value else "Descending Ordering"
    
    # Copies journeys_data
    journeys_data_copy = journeys_data.copy()
    
    # Creates matcher progress bar
    matcher_progress_v4 = widgets.IntProgress(
        value = 0,
        min = 0,
        max = len(journeys_data_copy["journey_id"]) - 1,
        step = 1,
        description = "Calculating v4.0",
        bar_style = "success",
        orientation = "horizontal",
        layout = Layout(width = "40%"),
        style = {"description_width":"initial"}
    )
    
    # Sorting function
    def sorting(array) :
        return array.sort_values(by = ["start_end"], ascending = ascending.value)
    
    matcher_v0(journeys_data_copy, matcher_progress_v4, sorting, out_matcher_v4)
        
# Runs all matcher versions
def matcher(_) :
    matcher_v1(_)
    matcher_v2(_)
    matcher_v3(_)
    matcher_v4(_)
    
# Displays widgets
widgets.VBox([ascending])

In [ ]:
# Creates run matcher buttons and corresponding outputs
matcher_button_v1 = widgets.Button(description = "Run Matcher v1.0", layout = Layout(width = "150px"))
out_matcher_v1 = widgets.Output()
matcher_button_v2 = widgets.Button(description = "Run Matcher v2.0", layout = Layout(width = "150px"))
out_matcher_v2 = widgets.Output()
matcher_button_v3 = widgets.Button(description = "Run Matcher v3.0", layout = Layout(width = "150px"))
out_matcher_v3 = widgets.Output()
matcher_button_v4 = widgets.Button(description = "Run Matcher v4.0", layout = Layout(width = "150px"))
out_matcher_v4 = widgets.Output()

matcher_button = widgets.Button(description = "Run All Matchers", layout = Layout(width = "150px"))

# Set run matcher button with function
matcher_button_v1.on_click(matcher_v1)
matcher_button_v2.on_click(matcher_v2)
matcher_button_v3.on_click(matcher_v3)
matcher_button_v4.on_click(matcher_v4)

matcher_button.on_click(matcher)

# Displays widgets
widgets.VBox([widgets.HBox([matcher_button, matcher_button_v1, matcher_button_v2, matcher_button_v3, matcher_button_v4]), 
              widgets.VBox([out_matcher_v1, out_matcher_v2, out_matcher_v3, out_matcher_v4])])

## Display and save last run Matcher version

In [ ]:
# Creates matcher display and matcher save buttons with corresponding outputs
matcher_display_button = widgets.Button(description = "Display Results", layout = Layout(width = "150px"))
matcher_save_button = widgets.Button(description = "Save Results", layout = Layout(width = "150px"))
out_matcher_display = widgets.Output()

def matcher_display(_) :
    with out_matcher_display :
        # Clears printed output
        clear_output()
        
        # Makes sure matcher is run first
        if matches is None :
            print("Run Matcher First")
            return
        
        # Prints out last matcher run
        print("Last Matcher Run: {0} {1}\n".format(last_matcher_run, asc_desc))
        
        # Prints guaranteed TPs who are leftover
        print("Guaranteed TP(s) Left:")
        unmatched_TPs = TP_pstats.where(~TP_pstats["TP"].isin(matches["TP"])).dropna(how = "all")
        for TP in unmatched_TPs.where(unmatched_TPs["guaranteed"] < 0).dropna(how = "all")["TP"].sort_values() :
            print(TP)

        # Prints journeys that are unmatched
        print("\nUnmatched Journeys:")
        unmatched_journeys = matches.where(matches["TP"].isnull()).dropna(how = "all")
        for journey_id in unmatched_journeys["journey_id"].sort_values() :
            print(journey_id)
            
        # Prints match data header
        print("\n{0:15} {1:15} {2:25} {3:15} {4:15}".format("Journey ID", "Journey Min", "Provider Given", "Guaranteed", "Distance"))
        for i,match in matches.iterrows() :
            # Formats TP name, guaranteed status, and distance
            if match["TP"] is not None :
                # Gets TP data from match
                TP_data = TP_pstats.where(TP_pstats["TP"] == match["TP"]).dropna(how = "all").iloc[0]
                TP = TP_data["TP"]
                guaranteed = "Yes" if int(TP_data["guaranteed"]) < 0 else "No"
                distance = "{:.2f}".format(iarray[np.argwhere(np.array(TP_pstats["TP"]) == TP_data["TP"])][0][0][i])
            else :
                TP = "-"
                guaranteed = "-"
                distance = "-"

            # Prints match data
            print("{0:15} {1:15} {2:25} {3:15} {4:15}".format(match["journey_id"], 
                                                       "{:.2f}".format(np.min(iarray[:,i][np.nonzero(iarray[:,i])])) if len(iarray[:,i][np.nonzero(iarray[:,i])]) > 0 else "-",
                                                       TP,
                                                       guaranteed,
                                                       distance))
            
def matcher_save(_) :
    with out_matcher_display :
        # Clears output
        clear_output()
        
        # Makes sure matcher is run first
        if matches is None :
            print("Run Matcher First")
            return
        
        # Prints out last matcher run
        print("Last Matcher Run: {0} {1}\n".format(last_matcher_run, asc_desc))
        
        print("Saving Matcher Results", end = "\r")
        
        # Opens and clears Matcher Sheet's Results workheet
        results_sheet = gs.open_by_url(matcher_sheet_url).worksheet("Results")
        results_sheet.clear()
        
        # Initializes output from matcher
        columns = ["Reg", "TP", "Journey ID", "Start Lat", "Start Lng", "Start End", "Men", "Tail Lift", "Max m3", "Price", "Journey Min", "Provider Given", "Guaranteed", "Distance", "Provider Min", "Increase On Min"]
        matcher_output = pd.DataFrame(columns = columns)
        
        for i,match in matches.iterrows() :
            # Gets journey data from match
            journey = data.where(data["journey_id"] == match["journey_id"]).dropna(how = "all").iloc[0]
            # Gets minimum iarray value for the journey
            journey_min = np.min(iarray[:,i][np.nonzero(iarray[:,i])]) if len(iarray[:,i][np.nonzero(iarray[:,i])]) > 0 else "-"
            
            # Formats TP name, TP registration, TP nickname, guaranteed status, and distance
            if match["TP"] is not None :
                # Gets TP data from match
                TP_data = TP_pstats.where(TP_pstats["TP"] == match["TP"]).dropna(how = "all").iloc[0]
                TP = TP_data["TP"]
                reg = TP.split("#")[1].strip()
                nickname = TP.split("#")[0].strip()
                guaranteed = TP_data["guaranteed"]
                
                iarray_TP = iarray[np.argwhere(np.array(TP_pstats["TP"]) == TP_data["TP"])][0][0]
                provider_min = np.min(iarray_TP[np.nonzero(iarray_TP)])
                distance = iarray_TP[i]
            else :
                TP = "-"
                reg = "-"
                nickname = "-"
                guaranteed = "-"
                provider_min = "-"
                distance = "-"
            
            # Saves match information in output from matcher
            matcher_output = matcher_output.append(pd.DataFrame([[reg, nickname, match["journey_id"], journey["start_lat"], journey["start_lng"], journey["start_end"], journey["number_of_men"], journey["tail_lift"], journey["max_m3"], journey["tp_amount"], journey_min, TP, guaranteed, distance, provider_min, (distance - journey_min) if journey_min != "-" and distance != "-" else "-"]], columns = columns))
        
        # Saves output from matcher to Matcher Sheet's Results worksheet
        results_sheet.update([matcher_output.columns.values.tolist()] + matcher_output.values.tolist(), raw = False)
        
        # Gets unmatched guaranteed TPs
        unmatched_TPs = TP_pstats.where(np.logical_and(~TP_pstats["TP"].isin(matches["TP"]), TP_pstats["guaranteed"] < 0)).dropna(how = "all")
        # Saves unmatched guaranteed TPs to Matcher Sheet's Results worksheet
        results_sheet.update("R1", "Left TPs")
        results_sheet.update("R2:R", [[TP] for TP in unmatched_TPs["TP"]], raw = False)
        
        print("Saved Matcher Results              ")
        # Prints Matcher Sheet url
        print("Go to: {0} to view".format(matcher_sheet_url))
        
# Sets matcher display and matcher save buttons with their corresponding functions
matcher_display_button.on_click(matcher_display)
matcher_save_button.on_click(matcher_save)

# Displays widgets
widgets.VBox([widgets.HBox([matcher_display_button, matcher_save_button]), out_matcher_display])

# Done